In [1]:
import numpy as np
import os 
import h5py
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import savgol_filter
from sklearn.preprocessing import MinMaxScaler
import warnings
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import SplineTransformer, QuantileTransformer
from scipy import stats
import scipy.stats
import seaborn as sns
import tensorflow as tf
import scipy.io
from tqdm import tqdm
import pickle
import time

/Users/akhilbandi/anaconda3/envs/GLM_tensorflow/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import encoding_GLM as glm

In [3]:
print(tf.config.experimental.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
from itertools import accumulate
from collections import Counter

def get_data_from_each_CV_fold(train_directory, test_directory, fold):
    directory_train = train_directory.format(fold)
    os.chdir(directory_train)

    behav = scipy.io.loadmat('behav_big_matrix.mat')
    behav_ids = scipy.io.loadmat('behav_big_matrix_ids.mat')

    behav_matrix = behav['behav_big_matrix']
    behav_ids_matrix = behav_ids['behav_big_matrix_ids'][0]

    response = scipy.io.loadmat('combined_response.mat')
    response_matrix = response['combined_response']
    response_matrix[response_matrix > 0.05] = 1

    X_train = behav_matrix
    Y_train = response_matrix
    
    behav_IDS = []
    for trial in list(range(behav_ids['behav_big_matrix_ids'][0].shape[0])):
        behav_IDS.append(behav_ids['behav_big_matrix_ids'][0][trial][0])

    # Count the occurrences of each element in the list
    counter = Counter(behav_IDS)

    # Get the unique values
    unique_values = list(counter.keys())

    # Get the count of each unique value
    count_of_values = list(counter.values())

    IDS_index = np.array(list(accumulate(count_of_values)))-1
    IDS_for_count_of_values = []
    for index in IDS_index[0:418]:
        IDS_for_count_of_values.append(behav_IDS[index])
        
    directory_test = test_directory.format(fold)
    os.chdir(directory_test)

    behav = scipy.io.loadmat('behav_big_matrix.mat')
    behav_ids = scipy.io.loadmat('behav_big_matrix_ids.mat')

    behav_matrix = behav['behav_big_matrix']
    behav_ids_matrix = behav_ids['behav_big_matrix_ids'][0]

    response = scipy.io.loadmat('combined_response.mat')
    response_matrix = response['combined_response']
    response_matrix[response_matrix > 0.05] = 1

    X_test = behav_matrix
    Y_test = response_matrix
    
    # Clean up design matrix and z-score along sample dimension
    X_train = X_train.T
    # Multiply deconvolved activity by 10 to mimic spike number
    Y_train = 10 * Y_train.T

    X_test = X_test.T
    # Multiply deconvolved activity by 10 to mimic spike number
    Y_test = 10 * Y_test.T
    
    return X_train, Y_train, X_test, Y_test, count_of_values, IDS_for_count_of_values

In [5]:
def train_model_on_cv_fold(X_train, Y_train, count_of_values):
    # Reset keras states
    tf.keras.backend.clear_session()

    model = glm.GLM(activation = 'exp', loss_type = 'poisson', 
                    regularization = 'elastic_net', lambda_series = 10.0 ** np.linspace(-1, -8, 10), 
                    l1_ratio = 0.98, smooth_strength = 0., 
                    optimizer = 'adam', learning_rate = 1e-3)
    
    model.fit(X_train, Y_train, feature_group_size = count_of_values, verbose = False) 
    
    return model

def train_model_on_cv_fold_GPU(X_train, Y_train, count_of_values):
    # Ensure the function runs on GPU
    with tf.device('/GPU:0'):
        # Reset keras states
        tf.keras.backend.clear_session()

        model = glm.GLM(activation = 'exp', loss_type = 'poisson', 
                        regularization = 'elastic_net', lambda_series = 10.0 ** np.linspace(-1, -8, 10), 
                        l1_ratio = 0.98, smooth_strength = 0., 
                        optimizer = 'adam', learning_rate = 1e-3)
        # Capture the start time
        start_time = time.time()
        
        # Fit the model
        model.fit(X_train, Y_train, feature_group_size=count_of_values, verbose=False) 
        
        # Capture the end time
        end_time = time.time()

    print(f"Time taken on GPU: {end_time - start_time:.4f} seconds")
    
    return model


In [7]:
for dataset in ['PPC B67 0629', 'PPC B67 0716', 'PPC B68 0628', 'PPC B68 0630', 'PPC B68 0703', 'PPC B77 0828', 
                'A1 B67 0703', 'A1 B68 0702', 'A1 B75 0908','A1 B77 0905']:
    directory = "/Volumes/Runyan2/Akhil/Runyan-Piasini-2017-master/Data/{}/all_state_model".format(dataset) 
    frac_dev_expl_all = []
    for fold_number in list(range(0,10)):
        train_directory = directory + "/prepost trial cv 73 #{}".format(fold_number+1)
        test_directory = directory + "/prepost trial cv 73 #{}/test".format(fold_number+1)

        for i in tqdm(range(5)):
            [X_train, Y_train, X_test, Y_test, count_of_values, IDS_for_count_of_values] = get_data_from_each_CV_fold(train_directory,
                                                                                                                      test_directory,
                                                                                                                      fold_number+1)
        model_trained = train_model_on_cv_fold_GPU(X_train, Y_train, count_of_values)
        model_trained.select_model(X_test, Y_test, min_lambda = 0.0, make_fig = False)
        frac_dev_expl, dev_model, dev_null, dev_expl = model_trained.evaluate(X_test, Y_test, make_fig = False)
        B_weights = model_trained.selected_w
        intercept_weight = model_trained.selected_w0
        y_pred = model_trained.predict(X_test)
        selec_lambda = model_trained.selected_lambda
        
        model_data = {
            'frac_dev_expl': frac_dev_expl,
            'dev_model': dev_model,
            'dev_null': dev_null, 
            'dev_expl': dev_expl, 
            'B_weights': B_weights,
            'intercept_weight': intercept_weight,
            'y_pred': y_pred,
            'selec_lambda': selec_lambda
        }
        
        os.chdir(train_directory)

        with open('model_data_a95_GPU.pkl', 'wb') as file:
            pickle.dump(model_data, file)
            
        frac_dev_expl_all.append(model_data['frac_dev_expl'])
    
    frac_dev_expl_all_state1 = np.array(frac_dev_expl_all, dtype=object)
    save_directory = "/Volumes/Runyan2/Akhil/Runyan-Piasini-2017-master/Data/{}".format(dataset)
    os.chdir(save_directory)
    np.save('performance_allstate_a98_GPU.npy', frac_dev_expl_all_state1)
    print(dataset + ' - DONE')

100%|██████████| 5/5 [00:01<00:00,  3.46it/s]


Time taken on GPU: 286.9554 seconds
Fraction deviance explained: mean = 0.1284, median = 0.0691


100%|██████████| 5/5 [00:01<00:00,  3.51it/s]


Time taken on GPU: 287.4806 seconds
Fraction deviance explained: mean = -0.0167, median = 0.0779


100%|██████████| 5/5 [00:01<00:00,  3.37it/s]


...

Time taken on GPU: 2207.6090 seconds
Fraction deviance explained: mean = 0.0795, median = 0.0475


100%|██████████| 5/5 [00:02<00:00,  2.20it/s]


Time taken on GPU: 2182.1687 seconds
Fraction deviance explained: mean = 0.0726, median = 0.0518


100%|██████████| 5/5 [00:02<00:00,  2.30it/s]


KeyboardInterrupt: 